In [1]:
import nltk
import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime
from nltk.corpus import stopwords
#from gensim import corpora, models, similarities

Exploratory text analysis of the transcript of a single cnn broadcast.

In [2]:
url = "http://transcripts.cnn.com/TRANSCRIPTS/2016.08.15.html"
res = requests.get(url)
txt = res.text
soup = BeautifulSoup(txt,"lxml")


In [3]:
shows = [show.get_text() for show in soup.find_all('div',class_='cnnTransDate')]
sections = soup.find_all('div',class_='cnnSectBulletItems')
section_links = [section.find_all('a') for section in sections]


In [4]:
transcript_links = [link['href'] for section in section_links for link in section]
transcript_links = transcript_links[0:40] # unicode error on the last element

In [5]:
base_url = "http://transcripts.cnn.com"
transcript_urls = [base_url+link for link in transcript_links]


In [6]:
res = requests.get(transcript_urls[0])
text = res.text

In [7]:
soup = BeautifulSoup(text,'lxml')
transcript = soup.find_all('p',class_="cnnBodyText")[2].text.replace(",","")

In [8]:
# Split by commercial break
p = re.compile("((COMMERCIAL BREAK))")
segments = re.split(p,transcript)
segments = [s for s in segments if s !="COMMERCIAL BREAK"]

In [9]:
# Chunk by speaker
words = [nltk.word_tokenize(seg) for seg in segments]
pos = [nltk.pos_tag(word) for word in words]
grammar = grammar = "speaker: {<NNP>+<:>}" #""" NP: {<NNP>+:}"""
cp = nltk.RegexpParser(grammar)
speakers = [cp.parse(seg) for seg in pos]

In [19]:
speakers[0:10]

[Tree('S', [(u'[', 'NN'), (u'20:00:21', 'CD'), (u']', 'CD'), Tree('speaker', [(u'ANDERSON', 'NNP'), (u'COOPER', 'NNP'), (u'CNN', 'NNP'), (u'ANCHOR', 'NNP'), (u':', ':')]), (u'Good', 'NNP'), (u'evening', 'NN'), (u'.', '.'), (u'Thanks', 'NNS'), (u'for', 'IN'), (u'joining', 'VBG'), (u'us', 'PRP'), (u'.', '.'), (u'Tonight', 'NNP'), (u'Donald', 'NNP'), (u'Trump', 'NNP'), (u'on', 'IN'), (u'prompter', 'NN'), (u'and', 'CC'), (u'on', 'IN'), (u'message', 'NN'), (u'for', 'IN'), (u'a', 'DT'), (u'second', 'JJ'), (u'straight', 'NN'), (u'Monday', 'NNP'), (u'this', 'DT'), (u'time', 'NN'), (u'laying', 'VBG'), (u'out', 'RP'), (u'his', 'PRP$'), (u'plan', 'NN'), (u'for', 'IN'), (u'defeating', 'NN'), (u'ISIS', 'NNP'), (u'.', '.'), (u'And', 'CC'), (u'for', 'IN'), (u'a', 'DT'), (u'second', 'JJ'), (u'straight', 'NN'), (u'Monday', 'NNP'), (u'Republicans', 'NNP'), (u'holding', 'VBG'), (u'their', 'PRP$'), (u'breath', 'NN'), (u'hoping', 'VBG'), (u'tomorrow', 'NN'), (u'wo', 'MD'), (u"n't", 'RB'), (u'be', 'VB'), (u

In [11]:
for tree in speakers[0].subtrees():
    if tree.label()=="speaker":
        print(tree)

(speaker ANDERSON/NNP COOPER/NNP CNN/NNP ANCHOR/NNP :/:)
(speaker
  BEGIN/NNP
  VIDEOTAPE/NNP
  )/NNP
  SARA/NNP
  MURRAY/NNP
  CNN/NNP
  POLITICS/NNP
  REPORTER/NNP
  (/NNP
  voice-over/NNP
  )/NNP
  :/:)
(speaker
  DONALD/NNP
  TRUMP/NNP
  (/NNP
  R/NNP
  )/NNP
  PRESIDENTIAL/NNP
  NOMINEE/NNP
  :/:)
(speaker MURRAY/NNP :/:)
(speaker TRUMP/NNP :/:)
(speaker MURRAY/NNP :/:)
(speaker TRUMP/NNP :/:)
(speaker MURRAY/NNP :/:)
(speaker TRUMP/NNP :/:)
(speaker MURRAY/NNP :/:)
(speaker TRUMP/NNP :/:)
(speaker MURRAY/NNP :/:)
(speaker TRUMP/NNP :/:)
(speaker COOPER/NNP :/:)
(speaker MURRAY/NNP :/:)
(speaker Mike/NNP Pence/NNP --/:)
(speaker COOPER/NNP :/:)
(speaker Sara/NNP Murray/NNP --/:)
(speaker
  COREY/NNP
  LEWANDOWSKI/NNP
  CNN/NNP
  POLITICAL/NNP
  COMMENTATOR/NNP
  :/:)
(speaker COOPER/NNP :/:)
(speaker
  REP./NNP
  SETH/NNP
  MOULTON/NNP
  (/NNP
  D/NNP
  )/NNP
  MASSACHUSETTS/NNP
  :/:)
(speaker COOPER/NNP :/:)
(speaker MOULTON/NNP :/:)
(speaker COOPER/NNP :/:)
(speaker
  CLARISSA/

In [12]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = sent_detector.tokenize(transcript)


In [20]:
words = [nltk.word_tokenize(sent) for sent in sentences]
pos = [nltk.pos_tag(word) for word in words]

In [16]:
stopwords = stopwords.words('english')

In [17]:
grammar = "speaker: {<NNP>+<:>}" #""" NP: {<NNP>+:}"""
cp = nltk.RegexpParser(grammar)

In [22]:
cp.parse(pos[0])

LookupError: 

===========================================================================
NLTK was unable to find the gs file!
Use software specific configuration paramaters or set the PATH environment variable.
===========================================================================

Tree('S', [(u'[', 'NN'), (u'20:00:21', 'CD'), (u']', 'CD'), Tree('speaker', [(u'ANDERSON', 'NNP'), (u'COOPER', 'NNP'), (u'CNN', 'NNP'), (u'ANCHOR', 'NNP'), (u':', ':')]), (u'Good', 'NNP'), (u'evening', 'NN'), (u'.', '.')])

In [21]:
transcript

u'[20:00:21] ANDERSON COOPER CNN ANCHOR:  Good evening.  Thanks for joining us.   Tonight Donald Trump on prompter and on message for a second straight Monday this time laying out his plan for defeating ISIS.  And for a second straight Monday Republicans holding their breath hoping tomorrow won\'t be another Tuesday like last Tuesday when Trump got off message and to many campaign watchers stayed that way all week.   So this afternoon under pressure from the party and trailing in a new batch of polling in key battleground states Trump stepped to the microphone looked into the teleprompter and started reading.   CNN\'s Sara Murray reports. (BEGIN VIDEOTAPE) SARA MURRAY CNN POLITICS REPORTER (voice-over):  Donald Trump battling to take control of his campaign message by laying out his vision to defeat ISIS.   DONALD TRUMP (R) PRESIDENTIAL NOMINEE:  We cannot let this evil continue.   MURRAY:  Trump looking to rebound from a rocky stretch and ginning up doubts about Hillary Clinton\'s for